### Package Imports

In [ ]:
import os
import time
import numpy as np
import tensorflow as tf
import spectral
import matplotlib.pyplot as plt
from utils import Data, PreProcessing
from random import randint, uniform
from sacred import Experiment
from sacred.observers import FileStorageObserver
from scipy.io import loadmat
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.utils import to_categorical      
from keras.layers import SeparableConv2D, Flatten, Dense, Dropout, Input, Concatenate, Add
from keras.models import Model, load_model
from keras.callbacks import ModelCheckpoint
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, cohen_kappa_score, balanced_accuracy_score

In [ ]:
# Allow tensorflow to utilise more memory 
gpus = tf.config.list_physical_devices('GPU')
if gpus: 
    tf.config.set_logical_device_configuration(
        gpus[0],
        [tf.config.LogicalDeviceConfiguration(memory_limit=11000)]  # 11GB
    )

### Experiment Configurations

In [ ]:
# Create the experiment
ex_name = '2SRS'
ex = Experiment(ex_name, interactive=True)

In [ ]:
# Define the global variables
configs = {
    'dataset_short_name': 'IP',  # IP, PU, SA
    'include_zero_labels': False,
    'window_size': 25,
    'window_size_s': 5,
    'test_size': 0.9,
    'n_components': 30,  # number of components for PCA
    'normalize_samples': False,
    'random_state': 792,  #randint(0, 1000) , IP=792, PU=477, SA=468
    'perform_oversampling': False,
    'apply_data_augmentation': False,
    'da_max_samples': 1500,  # Maximum number of samples per class (for data augmentation)
}

### Load Raw Data

In [ ]:
# Load the data
data, labels, n_classes, dataset_name, rgb_bands = Data.load_data(configs['dataset_short_name'])
print(f'Data shape is :{data.shape}')
print(f'Labels shape is: {labels.shape}')

In [ ]:
# Decrease the samples and number of classes if needed
if not configs['include_zero_labels']:
    n_classes -= 1

### Band Normalization & Reduction (Spectral Dimension Reduction)

In [ ]:
# First, we resize the data to 2D for band reduction (PCA or Auto-Encoder)
original_shape = data.shape
bands = original_shape[2]
data = data.reshape((-1, data.shape[2]))

In [ ]:
# Band reduction process
bands = configs['n_components']
minmax_scaler = MinMaxScaler()
data = minmax_scaler.fit_transform(data)
pca = PCA(n_components=configs['n_components'])
data = pca.fit_transform(data)

print(f'Data shape is :{data.shape}')
print(f'Labels shape is: {labels.shape}')

In [ ]:
# Reshape the data back to 3D
data = data.reshape((original_shape[0], original_shape[1], bands))
data.shape
print(f'Data shape is :{data.shape}')
print(f'Labels shape is: {labels.shape}')

### Create Image Patches (For Spatial) and Flattened Image Patches (For Spectral)

In [ ]:
# Create image patches/cubes
X_2d, y_2d = PreProcessing.create_image_cubes(data, labels, window_size=configs['window_size'], include_zero_labels=configs['include_zero_labels'])
X_2ds, y_2ds = PreProcessing.create_image_cubes(data, labels, window_size=configs['window_size_s'], include_zero_labels=configs['include_zero_labels'])

X_2ds.shape, X_2d.shape, y_2ds.shape, y_2d.shape

In [ ]:
# Count how many unique class and samples for each class
unique, counts = np.unique(y_2ds, return_counts=True)

unique, counts

### Train-Test Split

In [ ]:
# Reshape the Spectral data
X_2ds = X_2ds.reshape((X_2ds.shape[0], X_2ds.shape[1]*X_2ds.shape[2], X_2ds.shape[3], 1))
X_2ds.shape, X_2d.shape, y_2ds.shape, y_2d.shape

In [ ]:
# Split the 1D data and 2D data into training and testing sets
X_2ds_train, X_2ds_test, y_train, y_test = train_test_split(X_2ds, y_2ds, test_size=configs['test_size'], random_state=configs['random_state'],
                                                            stratify=y_2ds)
X_2d_train, X_2d_test, y_train2, y_test2 = train_test_split(X_2d, y_2d, test_size=configs['test_size'], random_state=configs['random_state'],
                                                            stratify=y_2d)

# Check that the split is exactly the same
print((y_train == y_train2).all())
print((y_test == y_test2).all())

# Reshape y2d
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

X_2ds_train.shape, X_2ds_test.shape, X_2d_train.shape, X_2d_test.shape, y_train.shape, y_test.shape

### Training Data Augmentation

In [ ]:
# Perform data augmentation
class_weights = None
if configs['apply_data_augmentation']:
    unique, counts = np.unique(y_train.argmax(axis=1), return_counts=True)
    print(unique)
    print(counts)
    max_sample = max(counts) if max(counts) > configs['da_max_samples'] else configs['da_max_samples']
    print(f'Max no. of samples: {max_sample}')
    for label, sample_count in zip(unique, counts):
        required_samples = max_sample - sample_count
        new_X_2d = np.zeros((required_samples, X_2d_train.shape[1], X_2d_train.shape[2], X_2d_train.shape[3]))
        new_X_2ds = np.zeros((required_samples, X_2ds_train.shape[1], X_2ds_train.shape[2], X_2ds_train.shape[3]))
        new_y = np.zeros((required_samples,))
        for i in range(required_samples):
            # Get the samples for the current label in the loop
            samples_2d = X_2d_train[y_train.argmax(1) == label]
            samples_2ds = X_2ds_train[y_train.argmax(1) == label]
            # Select a random sample to perform data augmentation
            sample_index = randint(0, samples_2d.shape[0]-1)
            chosen_X_2d = samples_2d[sample_index]
            chosen_X_2ds = samples_2ds[sample_index]
            # Select a random data augmentation method
            augmentation_method = randint(0, 4)
            # Augment the data
            if augmentation_method == 0:
                new_X_2d[i] = np.rot90(chosen_X_2d, 1)
            elif augmentation_method == 1:
                new_X_2d[i] = np.rot90(chosen_X_2d, 2)
            elif augmentation_method == 2:
                new_X_2d[i] = np.rot90(chosen_X_2d, 3)
            elif augmentation_method == 3:
                new_X_2d[i] = np.flip(chosen_X_2d, 0)
            else:
                new_X_2d[i] = np.flip(chosen_X_2d, 1)
            # Randomize row and assign it as new X
            new_X_2ds[i] = np.copy(chosen_X_2ds)
            np.random.shuffle(new_X_2ds[i])
            new_y[i] = label

        # Combine the new samples with the original ones
        X_2d_train = np.concatenate((X_2d_train, new_X_2d))
        X_2ds_train = np.concatenate((X_2ds_train, new_X_2ds))
        y_train = y_train.argmax(1)
        y_train = np.concatenate((y_train, new_y))
        y_train = to_categorical(y_train)

X_2d_train.shape, X_2ds_train.shape, y_train.shape

### Build the 2SRS Model

In [ ]:
def sep2d_residual_block(input_layer, filters, kernel_size):
    first_layer = SeparableConv2D(filters=filters, kernel_size=kernel_size,
                                  activation='relu', padding='same')(input_layer)
    x = SeparableConv2D(filters=filters, kernel_size=kernel_size, activation='relu', padding='same')(first_layer)
    x = SeparableConv2D(filters=filters, kernel_size=kernel_size, activation='relu', padding='same')(x)
    x = Add()([x, first_layer])
    
    return x

In [ ]:
# # Building the Spectral 2D conv model
# input2ds_layer = Input((configs['window_size_s']*configs['window_size_s'], bands, 1))
# output2ds_layer = sep2d_residual_block(input2ds_layer, filters=64, kernel_size=(1, 5))
# output2ds_layer = sep2d_residual_block(output2ds_layer, filters=128, kernel_size=(1, 5))
# output2ds_layer = sep2d_residual_block(output2ds_layer, filters=256, kernel_size=(1, 5))
# output2ds_layer = Flatten()(output2ds_layer)
# output2ds_layer = Dense(units=256, activation='relu')(output2ds_layer)
# output2ds_layer = Dropout(0.4)(output2ds_layer)
# output2ds_layer = Dense(units=128, activation='relu')(output2ds_layer)
# output2ds_layer = Dropout(0.4)(output2ds_layer)

# # Compile the 1D conv model
# model_2ds = Model(inputs=input2ds_layer, outputs=output2ds_layer)
# model_2ds.summary()

In [ ]:
# # Building the Spatial 2D conv model
# input2d_layer = Input((configs['window_size'], configs['window_size'], X_2d_train.shape[3]))
# output2d_layer = sep2d_residual_block(input2d_layer, filters=64, kernel_size=(3, 3))
# output2d_layer = sep2d_residual_block(output2d_layer, filters=128, kernel_size=(3, 3))
# output2d_layer = sep2d_residual_block(output2d_layer, filters=256, kernel_size=(3, 3))
# output2d_layer = Flatten()(output2d_layer)
# output2d_layer = Dense(units=256, activation='relu')(output2d_layer)
# output2d_layer = Dropout(0.4)(output2d_layer)
# output2d_layer = Dense(units=128, activation='relu')(output2d_layer)
# output2d_layer = Dropout(0.4)(output2d_layer)

# # Compile the 1D conv model
# model_2d = Model(inputs=input2d_layer, outputs=output2d_layer)
# model_2d.summary()

In [ ]:
# # Concatenate the 1DConv model with the 2DConv model
# output_layer = Concatenate()([output2ds_layer, output2d_layer])

# # Add some dense layers after concatenating
# output_layer = Dense(units=256, activation='relu')(output_layer)
# output_layer = Dropout(0.4)(output_layer)
# output_layer = Dense(units=128, activation='relu')(output_layer)
# output_layer = Dropout(0.2)(output_layer)
# output_layer = Dense(units=n_classes, activation='softmax')(output_layer)

# # Finalize the model
# model = Model(inputs=[model_2ds.input, model_2d.input], outputs=output_layer)
# model.summary()

In [ ]:
def build_2SRS_model(input_tensor_2ds=None, input_tensor_2d=None, spectral_window_size=configs['window_size_s'],
                     spatial_window_size=configs['window_size']):
    # Spectral stream
    if input_tensor_2ds != None:
        input_layer_2ds = Input(tensor=input_tensor_2ds)
    else:
        input_layer_2ds = Input((spectral_window_size**2, bands, 1))
    output_layer_2ds = sep2d_residual_block(input_layer_2ds, filters=64, kernel_size=(1, 5))
    output_layer_2ds = sep2d_residual_block(output_layer_2ds, filters=128, kernel_size=(1, 5))
    output_layer_2ds = sep2d_residual_block(output_layer_2ds, filters=256, kernel_size=(1, 5))
    output_layer_2ds = Flatten()(output_layer_2ds)
    output_layer_2ds = Dense(units=256, activation='relu')(output_layer_2ds)
    output_layer_2ds = Dropout(0.4)(output_layer_2ds)
    output_layer_2ds = Dense(units=128, activation='relu')(output_layer_2ds)
    output_layer_2ds = Dropout(0.4)(output_layer_2ds)

    # Spatial stream
    if input_tensor_2d != None:
        input_layer_2d = Input(tensor=input_tensor_2d)
    else:
        input_layer_2d = Input((spatial_window_size, spatial_window_size, bands))
    output_layer_2d = sep2d_residual_block(input_layer_2d, filters=64, kernel_size=(3, 3))
    output_layer_2d = sep2d_residual_block(output_layer_2d, filters=128, kernel_size=(3, 3))
    output_layer_2d = sep2d_residual_block(output_layer_2d, filters=256, kernel_size=(3, 3))
    output_layer_2d = Flatten()(output_layer_2d)
    output_layer_2d = Dense(units=256, activation='relu')(output_layer_2d)
    output_layer_2d = Dropout(0.4)(output_layer_2d)
    output_layer_2d = Dense(units=128, activation='relu')(output_layer_2d)
    output_layer_2d = Dropout(0.4)(output_layer_2d)

    # Concatenation of the two streams
    output_layer = Concatenate()([output_layer_2ds, output_layer_2d])
    output_layer = Dense(units=256, activation='relu')(output_layer)
    output_layer = Dropout(0.4)(output_layer)
    output_layer = Dense(units=128, activation='relu')(output_layer)
    output_layer = Dropout(0.2)(output_layer)
    output_layer = Dense(units=n_classes, activation='softmax')(output_layer)

    # Finalize the model
    model = Model(inputs=[input_layer_2ds, input_layer_2d], outputs=output_layer)

    return model

In [ ]:
def calculate_flops():
    import tensorflow as tf
    import keras.backend as K
    from keras.applications.mobilenet import MobileNet
    from keras.models import Model

    run_meta = tf.compat.v1.RunMetadata()
    with tf.compat.v1.Session(graph=tf.Graph()) as sess:
        K.set_session(sess)

        input_tensor_2ds = tf.compat.v1.placeholder('float32', shape=(1, configs['window_size_s'], bands, 1))
        input_tensor_2d = tf.compat.v1.placeholder('float32', shape=(1, configs['window_size'], configs['window_size'], bands))
        model = build_2SRS_model(input_tensor_2ds=input_tensor_2ds, input_tensor_2d=input_tensor_2d)

        opts = tf.compat.v1.profiler.ProfileOptionBuilder.float_operation()    
        flops = tf.compat.v1.profiler.profile(sess.graph, run_meta=run_meta, cmd='op', options=opts)

        opts = tf.compat.v1.profiler.ProfileOptionBuilder.trainable_variables_parameter()    
        params = tf.compat.v1.profiler.profile(sess.graph, run_meta=run_meta, cmd='op', options=opts)

        print("{:,} --- {:,}".format(flops.total_float_ops, params.total_parameters))


In [ ]:
# Calculate the model's FLOPs
# calculate_flops()  # Uncomment to calculate the flops

In [ ]:
# Build the model
model = build_2SRS_model(spectral_window_size=configs['window_size_s']**2, spatial_window_size=configs['window_size'])
model.summary()

In [ ]:
# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# The path to save the model weights
filename = f"2SRS_{dataset_name}_{int(100 - configs['test_size'] * 100)}.hdf5"
dir_path = '../../weights'
checkpoint_path = os.path.join(dir_path, filename)

# Define a model checkpoint
checkpoint = ModelCheckpoint(checkpoint_path, monitor='val_accuracy', save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [ ]:
# Train the model
t = time.time()
history = model.fit(x=[X_2ds_train, X_2d_train], y=y_train, batch_size=100, epochs=100, callbacks=callbacks_list,
                    validation_data=((X_2ds_test, X_2d_test), y_test))
t = time.time() - t
print(f"Time taken for 2SRS training on {dataset_name} with {configs['test_size']} test size: {t} s")

### Model Evaluation

In [ ]:
# Load the best version of the model
model.load_weights(checkpoint_path)

In [ ]:
# Predict the test data
t = time.time()
y_preds = model.predict([X_2ds_test, X_2d_test], verbose=1, batch_size=100)
t = time.time() - t
print(f"Time taken for 2SRS training on {dataset_name} with {configs['test_size']} test size: {t} s")

# Reshape to be evaluated
y_test = np.argmax(y_test, axis=1)
y_preds = np.argmax(y_preds, axis=1)

y_preds.shape

In [ ]:
# Get the confusion matrix
cmatrix = confusion_matrix(y_test, y_preds)

# Plot the confusion matrix
plt.matshow(cmatrix, cmap='tab20c')

In [ ]:
# Get the accuracy of each class
cmatrix.diagonal()/cmatrix.sum(axis=1)

In [ ]:
print(f"Random state is set to: {configs['random_state']}")

In [ ]:
# Get the OA, AA, and Kappa on the test data
overall_accuracy = accuracy_score(y_test, y_preds)
avg_accuracy = balanced_accuracy_score(y_test, y_preds)
cohen_kappa = cohen_kappa_score(y_test, y_preds)

print('Overall accuracy: %0.2f' % (overall_accuracy * 100))
print('Average accuracy: %0.2f' % (avg_accuracy * 100))
print('Cohen\'s Kappa: %0.2f' % (cohen_kappa * 100))

In [ ]:
# Plotting the accuracy and loss curve
fig, (ax1, ax2) = plt.subplots(1, 2)
fig.set_size_inches(15, 7)
ax1.set_title('Accuracy')
ax2.set_title('Loss')

ax1.plot(history.history['accuracy'])
ax2.plot(history.history['loss'])
plt.show()

In [ ]:
# Display the image using the first 3 features of PCA of the dataset
pca_bands = (0 , 1 ,2)
spectral.imshow(data, pca_bands, figsize=(7, 7))
spectral.save_rgb(r'C:\Users\Zharfan Adli\Desktop\output_pca_bands.png', data[:, :, [0, 1, 2]], colors=spectral.spy_colors)

In [ ]:
# Display the ground truth labels map
spectral.imshow(classes=labels, figsize=(7, 7))
spectral.save_rgb(r'C:\Users\Zharfan Adli\Desktop\output.png', labels, colors=spectral.spy_colors)

In [ ]:
# Predict the whole data

preds_all = model.predict([X_2ds, X_2d], verbose=1)
preds_all = np.argmax(preds_all, axis=1)
pred_map = np.zeros(labels.shape)

preds_all.shape, pred_map.shape

In [ ]:
# Create and display the prediction map
k = 0
for i in range(pred_map.shape[0]):
    for j in range(pred_map.shape[1]):
        if labels[i][j] != 0:
            pred_map[i][j] = preds_all[k] + 1
            k += 1

spectral.imshow(classes=pred_map, figsize=(7, 7))
spectral.save_rgb(r'C:\Users\Zharfan Adli\Desktop\output_preds.png', pred_map, colors=spectral.spy_colors)